In [ ]:
# Cargando variables de entorno
from dotenv import load_dotenv
import os

# Try different .env paths
env_path = os.path.join("..", ".env")
# env path using os path join

print(f"🔍 Trying .env path: {env_path}")

# Load environment variables
found = load_dotenv(env_path)
print(f"✅ .env file {'found' if found else 'not found'}")

# Verify loaded variables
print("\n🔎 Environment variables:")
for var in ["PROCESSED_DATA_DIR", "RAW_DATA_DIR", "CHECKPOINT_DIR"]:
    value = os.getenv(var)
    print(f"{var}: {'✅' if value else '❌'} {value}")

img_dir_train = os.path.join(os.getenv("PROCESSED_DATA_DIR"), 'train2014')
img_dir_val = os.path.join(os.getenv("PROCESSED_DATA_DIR"), 'val2014')
img_dir_test = os.path.join(os.getenv("PROCESSED_DATA_DIR"), 'test2014')
ann_coco_text = os.path.join(os.getenv("RAW_DATA_DIR"), 'cocotext.v2.zip')

print(ann_coco_text)
# Evaluamos modelo finetuneado  para ver si mejoro alguna metrica
from vpc3_proyecto.model_training.dataset_donut_model import DonutTextDatasetFromCocoTextV2Raw
from vpc3_proyecto.model_evaluation.utils import get_last_checkpoint_folder
import torch
import glob
from transformers import VisionEncoderDecoderModel, DonutProcessor

torch.cuda.empty_cache()
donut_checkpoint_dir = os.path.join(os.getenv("CHECKPOINT_DIR"),
                                    'donut-finetuned-coco2014')  #TODO: promptear al usuario detectando las carpetas dentro de checkpint directory para que elija que folder usar / que modelo guardado elegir
last_ckpt = get_last_checkpoint_folder(donut_checkpoint_dir)
manually_saved_folder = os.path.join(last_ckpt, "manually-saved")

processor = DonutProcessor.from_pretrained(manually_saved_folder)

model = VisionEncoderDecoderModel.from_pretrained(
    manually_saved_folder,
    device_map="auto",
    torch_dtype=torch.float16
)

test_dataset = DonutTextDatasetFromCocoTextV2Raw(
    img_dir_test,
    ann_coco_text,
    processor=processor,
    max_length=512
)
model.config.decoder_start_token_id = processor.tokenizer.pad_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id


In [ ]:
# TODO: ATTENTION VISUALIZATION